In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os 
os.chdir('/content/drive/My Drive/pSp_PrivacyEncoder')

Mounted at /content/drive


In [2]:
import numpy as np

from PIL import Image
from tqdm import tqdm

import torch
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms

from torch.utils.data import DataLoader, Dataset

import pytorch_ssim


In [3]:
#original_path = "image_datasets/test_images"
#processed_path = "image_datasets/test_inferenced_privacy0.3"
#processed_path = "image_datasets/test_blurred"
#processed_path = "image_datasets/test_mosaiced"
##############################################################################
BATCH_SIZE = 4
NUM_WORKERS = 4

IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP', '.tiff'
]


In [4]:
def is_image_file(filename):
  return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)
  
class LoadDataset(Dataset):
    def __init__(self, path, transforms=None):
        files_path = []
        assert os.path.isdir(path), "{} is not a valid directory".format(path)
        for root, dirs, files in sorted(os.walk(path)):
          for file_name in files:
            if is_image_file(file_name):
              file_path = os.path.join(root, file_name)
              files_path.append(file_path)

        files_path = sorted(files_path)
        self.files = files_path
        self.transforms = transforms

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        path = self.files[i]
        img = Image.open(path).convert('RGB')
        if self.transforms is not None:
            img = self.transforms(img)
        return img


In [5]:
def SSIM_calculation(ori_path, pro_path, batch_size, num_workers):
  image_transforms = transforms.Compose([transforms.Resize((256,256)),
                                        transforms.ToTensor(),
                                        transforms.CenterCrop(size=(112,112))])
  ori_dataset = LoadDataset(path=ori_path, transforms=image_transforms)
  pro_dataset = LoadDataset(path=pro_path, transforms=image_transforms)
  print("The number of input original images are {}, and processed images are {}".format(len(ori_dataset),len(pro_dataset)))

  ori_dataloader = torch.utils.data.DataLoader(
      dataset = ori_dataset,
      shuffle = False,
      batch_size = batch_size,
      drop_last = False,
      num_workers = num_workers
  )
  pro_dataloader = torch.utils.data.DataLoader(
      dataset = pro_dataset,
      shuffle = False,
      batch_size = batch_size,
      drop_last = False,
      num_workers = num_workers
  )
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  
  print("Running on {}\n".format(device))

  ssim_values = []
  try:
    with tqdm(zip(ori_dataloader, pro_dataloader),total=min(len(ori_dataloader),len(pro_dataloader))) as t:
      for batch in t:
        ssim_value = pytorch_ssim.ssim(img1=batch[0][0].to(device), img2=batch[1][0].to(device))
        ssim_values.append(ssim_value.cpu().numpy())
  except KeyboardInterrupt:
    t.close()
    raise
  
  ssim_mean = np.mean(ssim_values, axis=0)
  ssim_std = np.std(ssim_values)
  print("\n#SSIM is {:.2f}+-{:.2f}".format(ssim_mean,ssim_std))

  return ssim_mean, ssim_std


# DeepPrivacy

*   SSIM is 0.61+-0.09

In [ ]:
# Calculating SSIM value between original images with CIAGAN output images
original_path = "/content/drive/MyDrive/Image_datasets/celeba_align_20k_21k_aligned"
processed_path = "./image_datasets/DeepPrivacy_celeba_20k_21k_output/generated"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#The SSIM value between original and CIAGAN Output Images is: 


The number of input original images are 976, and processed images are 976
Running on cuda:0



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 244/244 [02:47<00:00,  1.46it/s]


#SSIM is 0.61+-0.09


# CIAGAN

*   SSIM is 0.65+-0.07



In [ ]:
# Calculating SSIM value between original images with CIAGAN output images
original_path = "image_datasets/CIAGAN_celeba_20k_21k_output/original"
processed_path = "image_datasets/CIAGAN_celeba_20k_21k_output/generated"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#The SSIM value between original and CIAGAN Output Images is: 0.65+-0.07


The number of input original images are 976, and processed images are 976
Running on cuda:0



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 244/244 [01:35<00:00,  2.56it/s]


#SSIM is 0.65+-0.07


# Inference CelebA
### Calculating SSIM value between aligned images with inferenced output images

* Privacy 0.0 SSIM is 0.72+-0.08
* Privacy 0.1 SSIM is 0.72+-0.08
* Privacy 0.2 SSIM is 0.73+-0.08
* Privacy 0.3 SSIM is 
* Privacy 0.4 SSIM is 
* Privacy 0.5 SSIM is 
* Privacy 0.6 SSIM is 
* Privacy 0.7 SSIM is 
* Privacy 0.8 SSIM is 
* Privacy 0.9 SSIM is 
* Privacy 1.0 SSIM is 

In [6]:
# Calculating SSIM value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.0/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.0/generated"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#The SSIM value between original and generated Images is: 


The number of input original images are 976, and processed images are 976
Running on cuda:0



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 244/244 [01:40<00:00,  2.44it/s]


#SSIM is 0.72+-0.08


In [7]:
# Calculating SSIM value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.1/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.1/generated"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#The SSIM value between original and generated Images is: 


The number of input original images are 270, and processed images are 270
Running on cuda:0



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 68/68 [00:27<00:00,  2.50it/s]


#SSIM is 0.72+-0.08


In [ ]:
# Calculating SSIM value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.2/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.2/generated"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#The SSIM value between original and generated Images is: 


The number of input original images are 976, and processed images are 976
Running on cuda:0



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 244/244 [02:09<00:00,  1.88it/s]


#SSIM is 0.73+-0.08


In [ ]:
# Calculating SSIM value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.4/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.4/generated"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#The SSIM value between original and generated Images is: 


In [ ]:
# Calculating SSIM value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.5/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.5/generated"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#The SSIM value between original and generated Images is: 


In [ ]:
# Calculating SSIM value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.6/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.6/generated"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#The SSIM value between original and generated Images is: 


In [ ]:
# Calculating SSIM value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.7/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.7/generated"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#The SSIM value between original and generated Images is: 


In [ ]:
# Calculating SSIM value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.8/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.8/generated"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#The SSIM value between original and generated Images is: 


In [ ]:
# Calculating SSIM value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.9/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_0.9/generated"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#The SSIM value between original and generated Images is: 


In [ ]:
# Calculating SSIM value between original images with CIAGAN output images
original_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_1.0/original"
processed_path = "image_datasets/pSp_celeba_20k_21k_output/inference_results_privacy_1.0/generated"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#The SSIM value between original and generated Images is: 


# Inference FFHQ
### Calculating SSIM value between aligned images with inferenced output images

* Privacy 0.0 SSIM is 0.63+-0.09
* ***Privacy 0.1 SSIM is 0.64+-0.09***
* Privacy 0.2 SSIM is 0.64+-0.09
* Privacy 0.3 SSIM is 0.66+-0.09
* Privacy 0.4 SSIM is 0.67+-0.09
* Privacy 0.5 SSIM is 0.67+-0.09
* Privacy 0.6 SSIM is 0.69+-0.09
* Privacy 0.7 SSIM is 0.69+-0.09
* Privacy 0.8 SSIM is 0.69+-0.09
* Privacy 0.9 SSIM is 0.70+-0.09
* Privacy 1.0 SSIM is 0.70+-0.09

In [ ]:
"""# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.0"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.0 SSIM is 0.59+-0.10"""
# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.0"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.0 SSIM is 0.63+-0.09

In [ ]:
"""# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.1"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.1 SSIM is 0.60+-0.10"""
# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.1"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.1 SSIM is 0.64+-0.09

In [ ]:
"""# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.2"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.2 SSIM is 0.61+-0.10"""
# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.2"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.2 SSIM is 0.64+-0.09

In [ ]:
"""# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.3"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.3 SSIM is 0.62+-0.10"""
# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.3"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.3 SSIM is 0.66+-0.09

In [ ]:
"""# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.4"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.4 SSIM is 0.63+-0.10"""
# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.4"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.4 SSIM is 0.67+-0.09

In [ ]:
"""# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.5"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.5 SSIM is 0.64+-0.10"""
# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.5"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.5 SSIM is 0.67+-0.09

In [ ]:
"""# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.6"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.6 SSIM is 0.63+-0.10"""
# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.6"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.6 SSIM is 0.69+-0.09

In [ ]:
"""# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.7"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.7 SSIM is 0.65+-0.10"""
# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.7"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.7 SSIM is 0.69+-0.09

In [ ]:
"""# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.8"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.8 SSIM is 0.64+-0.10"""
# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.8"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.8 SSIM is 0.69+-0.09

In [ ]:
"""# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_0.9"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.9 SSIM is 0.65+-0.10"""
# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_0.9"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 0.9 SSIM is 0.70+-0.09

In [ ]:
"""# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq5k_6k_aligned"
processed_path = "image_datasets/inference_results_privacy_1.0"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 1.0 SSIM is 0.64+-0.10"""
# Calculating SSIM value between aligned images with inferenced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_inference_results_privacy_1.0"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Privacy 1.0 SSIM is 0.70+-0.09

In [ ]:
### ffhq10k_11k ###
#Privacy 0.0 SSIM is 0.63+-0.09
#Privacy 0.1 SSIM is 0.64+-0.09
#Privacy 0.2 SSIM is 0.64+-0.09
#Privacy 0.3 SSIM is 0.66+-0.09
#Privacy 0.4 SSIM is 0.67+-0.09
#Privacy 0.5 SSIM is 0.67+-0.09
#Privacy 0.6 SSIM is 0.69+-0.09
#Privacy 0.7 SSIM is 0.69+-0.09
#Privacy 0.8 SSIM is 0.69+-0.09
#Privacy 0.9 SSIM is 0.70+-0.09
#Privacy 1.0 SSIM is 0.70+-0.09

### ffhq5k_6k ###
#Privacy 0.0 SSIM is 0.59+-0.10
#Privacy 0.1 SSIM is 0.60+-0.10
#Privacy 0.2 SSIM is 0.61+-0.10
#Privacy 0.3 SSIM is 0.62+-0.10
#Privacy 0.4 SSIM is 0.63+-0.10
#Privacy 0.5 SSIM is 0.64+-0.10
#Privacy 0.6 SSIM is 0.63+-0.10
#Privacy 0.7 SSIM is 0.65+-0.10
#Privacy 0.8 SSIM is 0.64+-0.10
#Privacy 0.9 SSIM is 0.65+-0.10
#Privacy 1.0 SSIM is 0.64+-0.10

# Blurred
### Calculating SSIM value between aligned images with blurred output images

* Blurring With Kernel Size=065 SSIM is 0.60+-0.08
* Blurring With Kernel Size=059 SSIM is 0.61+-0.08
* ***Blurring With Kernel Size=043 SSIM is 0.65+-0.08***
* Blurring With Kernel Size=039 SSIM is 0.66+-0.08
* Blurring With Kernel Size=033 SSIM is 0.68+-0.08
* Blurring With Kernel Size=031 SSIM is 0.69+-0.08

In [ ]:
"""# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq5k_6k_blurred_ksize_349/ffhq5k_6k_blurred_ori"
processed_path = "image_datasets/ffhq5k_6k_blurred_ksize_349/ffhq5k_6k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=349 SSIM is 0.50+-0.09"""
# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_349/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_349/ffhq10k_11k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=349 SSIM is 0.50+-0.09

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [02:01<00:00,  2.04it/s]


#SSIM is 0.50+-0.09


In [ ]:
"""# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq5k_6k_blurred_ksize_149/ffhq5k_6k_blurred_ori"
processed_path = "image_datasets/ffhq5k_6k_blurred_ksize_149/ffhq5k_6k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=149 SSIM is 0.51+-0.09"""

"""# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_149/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_149/ffhq10k_11k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=149 SSIM is """

In [ ]:
"""# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq5k_6k_blurred_ksize_099/ffhq5k_6k_blurred_ori"
processed_path = "image_datasets/ffhq5k_6k_blurred_ksize_099/ffhq5k_6k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=099 SSIM is 0.55+-0.08"""

"""# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_099/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_099/ffhq10k_11k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=099 SSIM is """

In [ ]:
"""# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq5k_6k_blurred_ksize_065/ffhq5k_6k_blurred_ori"
processed_path = "image_datasets/ffhq5k_6k_blurred_ksize_065/ffhq5k_6k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=065 SSIM is 0.59+-0.08"""
# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_065/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_065/ffhq10k_11k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=065 SSIM is 0.60+-0.08

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [02:14<00:00,  1.84it/s]


#SSIM is 0.60+-0.08


In [ ]:
# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_059/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_059/ffhq10k_11k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=059 SSIM is 

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [04:20<00:00,  1.05s/it]


#SSIM is 0.61+-0.08


In [ ]:
"""# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq5k_6k_blurred_ksize_049/ffhq5k_6k_blurred_ori"
processed_path = "image_datasets/ffhq5k_6k_blurred_ksize_049/ffhq5k_6k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=049 SSIM is 0.62+-0.07"""

"""# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_049/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_049/ffhq10k_11k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=049 SSIM is """

In [ ]:
# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_043/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_043/ffhq10k_11k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=043 SSIM is 0.65+-0.08

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [02:03<00:00,  2.00it/s]


#SSIM is 0.65+-0.08


In [ ]:
"""# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq5k_6k_blurred_ksize_039/ffhq5k_6k_blurred_ori"
processed_path = "image_datasets/ffhq5k_6k_blurred_ksize_039/ffhq5k_6k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=039 SSIM is 0.65+-0.07"""
# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_039/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_039/ffhq10k_11k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=039 SSIM is 0.66+-0.08

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [03:44<00:00,  1.10it/s]


#SSIM is 0.66+-0.08


In [ ]:
"""# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_033/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_033/ffhq10k_11k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=033 SSIM is 0.68+-0.08"""

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [02:03<00:00,  2.00it/s]


#SSIM is 0.68+-0.08


In [ ]:
# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_031/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_031/ffhq10k_11k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=031 SSIM is 

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [04:25<00:00,  1.08s/it]


#SSIM is 0.69+-0.08


In [ ]:
"""# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq5k_6k_blurred_ksize_029/ffhq5k_6k_blurred_ori"
processed_path = "image_datasets/ffhq5k_6k_blurred_ksize_029/ffhq5k_6k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=029 SSIM is 0.69+-0.07"""

"""# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_029/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_029/ffhq10k_11k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=029 SSIM is """

In [ ]:
"""# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq5k_6k_blurred_ksize_009/ffhq5k_6k_blurred_ori"
processed_path = "image_datasets/ffhq5k_6k_blurred_ksize_009/ffhq5k_6k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=009 SSIM is 0.86+-0.05"""

"""# Calculating SSIM value between aligned images with blurred output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_009/ffhq10k_11k_blurred_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_009/ffhq10k_11k_blurred"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Blurring With Kernel Size=009 SSIM is """

# Mosaiced

* Mosaicing With Neighbor=13 SSIM is 0.46+-0.08
* Mosaicing With Neighbor=09 SSIM is 0.51+-0.08
* Mosaicing With Neighbor=07 SSIM is 0.54+-0.08
* Mosaicing With Neighbor=05 SSIM is 0.62+-0.07
* ***Mosaicing With Neighbor=04 SSIM is 0.68+-0.07***
* Mosaicing With Neighbor=03 SSIM is 0.75+-0.06

In [ ]:
# Calculating SSIM value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor100/ffhq10k_11k_mosaiced_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor100/ffhq10k_11k_mosaiced"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor=100 SSIM is 0.40+-0.12

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [01:29<00:00,  2.76it/s]


#SSIM is 0.40+-0.12


In [ ]:
# Calculating SSIM value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor80/ffhq10k_11k_mosaiced_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor80/ffhq10k_11k_mosaiced"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor=80 SSIM is 

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [01:26<00:00,  2.87it/s]


#SSIM is 0.42+-0.10


In [ ]:
# Calculating SSIM value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor50/ffhq5k_6k_mosaiced_ori"
processed_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor50/ffhq5k_6k_mosaiced"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor=50 SSIM is 0.42+-0.10

In [ ]:
# Calculating SSIM value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor25/ffhq5k_6k_mosaiced_ori"
processed_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor25/ffhq5k_6k_mosaiced"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor=25 SSIM is 0.45+-0.09

In [ ]:
# Calculating SSIM value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor20/ffhq5k_6k_mosaiced_ori"
processed_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor20/ffhq5k_6k_mosaiced"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor=20 SSIM is 0.44+-0.08

In [ ]:
# Calculating SSIM value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor15/ffhq5k_6k_mosaiced_ori"
processed_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor15/ffhq5k_6k_mosaiced"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor=15 SSIM is 0.45+-0.08

In [ ]:
# Calculating SSIM value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor13/ffhq10k_11k_mosaiced_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor13/ffhq10k_11k_mosaiced"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor=13 SSIM is 0.46+-0.08

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [03:22<00:00,  1.22it/s]


#SSIM is 0.46+-0.08


In [ ]:
# Calculating SSIM value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor10/ffhq5k_6k_mosaiced_ori"
processed_path = "image_datasets/ffhq5k_6k_mosaiced_neighbor10/ffhq5k_6k_mosaiced"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor=10 SSIM is 0.48+-0.07

In [ ]:
# Calculating SSIM value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor09/ffhq10k_11k_mosaiced_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor09/ffhq10k_11k_mosaiced"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor=09 SSIM is 0.51+-0.08

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [02:07<00:00,  1.94it/s]


#SSIM is 0.51+-0.08


In [ ]:
# Calculating SSIM value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor07/ffhq10k_11k_mosaiced_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor07/ffhq10k_11k_mosaiced"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor=07 SSIM is 0.54+-0.08

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [02:07<00:00,  1.94it/s]


#SSIM is 0.54+-0.08


In [ ]:
# Calculating SSIM value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor05/ffhq10k_11k_mosaiced_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor05/ffhq10k_11k_mosaiced"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor=05 SSIM is 0.62+-0.07

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [02:08<00:00,  1.93it/s]



#SSIM is 0.62+-0.07


In [ ]:
# Calculating SSIM value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor04/ffhq10k_11k_mosaiced_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor04/ffhq10k_11k_mosaiced"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor=04 SSIM is 0.68+-0.07

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [02:02<00:00,  2.01it/s]


#SSIM is 0.68+-0.07


In [ ]:
# Calculating SSIM value between aligned images with mosaiced output images
original_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor03/ffhq10k_11k_mosaiced_ori"
processed_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_mosaiced_neighbor03/ffhq10k_11k_mosaiced"
ssim_mean, ssim_std = SSIM_calculation(ori_path=original_path, pro_path=processed_path, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
#Mosaicing With Neighbor=03 SSIM is 0.75+-0.06

The number of input original images are 988, and processed images are 988
Running on cuda:0



100%|██████████| 247/247 [02:07<00:00,  1.94it/s]


#SSIM is 0.75+-0.06
